# **Week 1 : Linear Algebra and Calculus**

In this assignment, we shall explore the concepts of analytic and numeric computation of gradients. Further, we will have a look at the computation graph which is a central concept to building a neural network. For learning how gradients are computed analytically, refer to the resources provided in this week.

<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQWsYeD8ZlYxFBB33qIn7bwQvP-KuvLZXJOoA&usqp=CAU"
 style="float:center;width:50px;height:50px;">

# **Importing Libraries**
Feel free to import any additional libraries required

In [38]:
# Import all libraries here

import numpy as np
import pandas as pd

# Setting the seed for reproducible results

# *Problem 1(a)*

In this problem, we shall be exploring the concepts of analytic and numeric computation of gradients for scalar valued functions. 

\begin{equation}
z = w^{T}x + b \\ 
\hat{y} = \sigma(z) = \frac{1}{1+e^{-z}}\\ 
L(\hat{y}, y) = y.log(\hat{y}) + (1-y).log(1-\hat{y})
\end{equation}

For this set of equations, the vector w, and scalars b, y are to be treated as constants. \\

Now, let us find the analytic gradient of the function L with respect to the function x. That is, compute $\frac{\delta L}{\delta x}$. Write the answer obtained as code in the function provided.

Initialise $w$ to a $10 \times 1$ vector sampled over a standard multivariate gaussian distribution, b to a uniform random variable over the interval $(0, 1)$, y to a bernoulli random variable over $\{0, 1\}$

In [39]:
# Initialisation : 
w = np.random.normal(size=(10, 1))
b = np.random.uniform(0,1)
y = np.random.binomial(0,1)

In [40]:
def analytic_grad(x) : 

    ### WRITE CODE BELOW ###
    #calculates the dot product of the transpose of w and x and adds b
    z = np.dot(w.T, x) + b
    #calculates the sigmoid function using z
    sigma = 1 / (1 + np.exp(-z))
    #calculates the gradient with respect to sigma
    analytic_gradient_wrt_sigma = (sigma - y) / (sigma * (1 - sigma))
     #calculates the gradient with respect to z
    analytic_gradient_wrt_z = sigma * (1 - sigma)
    analytic_gradient_wrt_x = w
     #finally calculates the gradient with respect to x
    analytic_gradient_wrt_x = np.dot(analytic_gradient_wrt_sigma * analytic_gradient_wrt_z, analytic_gradient_wrt_x)

    return analytic_gradient_wrt_x
    ### WRITE CODE ABOVE ###

# *Problem 1(b)*

Now, we compute the numeric gradient for the function L. Refer to [this](https://stackoverflow.com/questions/38854363/is-there-any-standard-way-to-calculate-the-numerical-gradient) stack exchange post to see how numeric gradients are computed

In [41]:
def numeric_grad(x) : 

    ### WRITE CODE BELOW ###

     # first we create x_plus_epsilon and x_minus_epsilon by adding and subtracting epsilon from x 
    epsilon=1e-5
    x_plus_epsilon = x + epsilon
    x_minus_epsilon = x - epsilon
    
    # Calculate the logistic loss function at x_plus_epsilon and x_minus_epsilon
    L_plus_epsilon = -y * np.log(1 / (1 + np.exp(-np.dot(w.T, x_plus_epsilon) - b))) - (1 - y) * np.log(1 - 1 / (1 + np.exp(-np.dot(w.T, x_plus_epsilon) - b)))
    L_minus_epsilon = -y * np.log(1 / (1 + np.exp(-np.dot(w.T, x_minus_epsilon) - b))) - (1 - y) * np.log(1 - 1 / (1 + np.exp(-np.dot(w.T, x_minus_epsilon) - b)))
    
    # now finally compute the gradient by taking the finite difference of L_plus_epsilon and L_minus_epsilon
    numeric_gradient = (L_plus_epsilon - L_minus_epsilon) / (2 * epsilon)
    
    return numeric_gradient

    ### WRITE CODE ABOVE ###
  

# *Problem 2*

Here, we'll be looking at computational graphs, and their calculus, finding gradients of  variables with respect to other variables in the graph.



We'll be looking at nodes of the graph that are operation based, i.e., each operation performed has a node associated with it.

We'll provide you with example implementations for three of the nodes, the *add* node, the *nth power* node and the *sine* node, you'll have to write the classes for all the other nodes.

# **Multi-input nodes**

In [42]:
class Add: 
  
  # A class to add multiple variables together
  def __init__(self, lst_names, lst_values):
    self.lst_names = lst_names # This numpy arr contains all the variable names that are to be added, with each variable name in datatype str.
                               # Scalar addition is taken care of in a separate node, we could have fit into this node but thought it might be easier if it wasn't.
                               # So cases like "a + 1" are to be done separately, and cases like "b + c + d + 4" can be done as "(b + c + d) + 4" or "b + c + (d + 4)", 
                               #since our scalar addition supports only one variable and one scalar, we'll get to that later
    self.lst_values = lst_values # This numpy arr contains all the variable values that are to be added, as scalars.

  def compute_output(self):
    return np.sum(self.lst_values) # This computes the sum of all the variables
  
  def compute_gradients(self):
    return dict(zip(self.lst_names, np.ones(len(self.lst_names)))) # This gives the gradient of the sum wrt to all the input variables, as a dictionary, will be used later


class Multiply: 
  
  #Everything's almost the same as the add class, but this deals with multiplication of more than 1 variables
  def __init__(self, lst_names, lst_values):
    self.lst_names = lst_names # This numpy arr contains all the variable names that are to be multiplied, with each variable name in datatype str.
                               # Scalar multiplication is taken care of in a separate node
    self.lst_values = lst_values # This numpy arr contains all the variable values that are to be multiplied, as scalars.

  def compute_output(self):
    # Write your code to compute the output of this operation
    return np.prod(self.lst_values)
  def compute_gradients(self):
      gradients = {}
      for i, name in enumerate(self.lst_names):
        temp = np.copy(self.lst_values)
        temp[i] = 1
        gradients[name] = np.prod(temp)
        return gradients
    # Write your code to create a dictionary, storing all the gradients of the output wrt to each input (In this case there is only a single input that matters)

# **Scalar multiplication/addition nodes**

In [43]:
class Add_Scalar: 
  
  # A class to add a variable with a scalar
  def __init__(self, lst_names, lst_values):
    self.lst_names = lst_names # This numpy arr contains all the variable names that are to be added, with each variable name in datatype str.
                               # This list must only have 2 elements, the first should be the string corresponding to the name of the variable, 
                               #and the second should be a string of the scalar value to be added.
    self.lst_values = lst_values # This numpy arr contains all the variable values that are to be added, as scalars.

  def compute_output(self):
    # Write your code to compute the output of this operation
    return self.lst_values[0]+self.lst_values[1]

  def compute_gradients(self):
    # Write your code to create a dictionary, storing all the gradients of the output wrt to each input (In this case there is only a single input that matters)
    return {self.lst_names[0]:1}


class Multiply_Scalar: 
  
  # A class to multiply a variable with a scalar
  def __init__(self, lst_names, lst_values):
    self.lst_names = lst_names # This numpy arr contains all the variable names that are to be multiplied, with each variable name in datatype str.
                               # This list must only have 2 elements, the first should be the string corresponding to the name of the variable, 
                               #and the second should be a string of the scalar value to be multiplied.
    self.lst_values = lst_values # This numpy arr contains all the variable values that are to be multiplied, as scalars.

  def compute_output(self):
    # Write your code to compute the output of this operation
     return self.lst_values[0]*self.lst_values[1]

  def compute_gradients(self):
    # Write your code to create a dictionary, storing all the gradients of the output wrt to each input (In this case there is only a single input that matters)
     return {self.lst_names[0]:self.lst_values[1]}

# **Nodes for special functions**

In [44]:
class Power:

  def __init__(self, name, value, exponent):
    self.name = name # Name of the variable to be exponentiated
    self.value = value # Value of the variable
    self.exponent = exponent # Value of the exponent
  
  def compute_output(self):
    return np.power(self.value, self.exponent)
  
  def compute_gradients(self):
    return {self.name : self.exponent*(np.power(self.value, (self.exponent - 1)))}

class Sine: 

  # A class to apply the sine function on a variable
  def __init__(self, name, value):
    self.name = name
    self.value = value
  
  def compute_output(self):
    return np.sin(self.value)
  
  def compute_gradients(self):
    return {self.name : np.cos(self.value)}


class Logarithm:

  # A class to compute the logarithm of a value
  def __init__(self, name, value):
    self.name = name
    self.value = value
  
  def compute_output(self):
    # Write your code here
    # Compute_output can be computed using np.log() function
    return np.log(self.value)

  def compute_gradients(self):
    # Write your code here
    #compute_gradients computed using derivatives of logarithms function
     return {self.name : 1/self.value}

class Exponential: 

  # A class to compute the exponential of a value
  def __init__(self, name, value):
    self.name = name
    self.value = value
  
  def compute_output(self):
    # Write your code here
    #compute_output() computed using np.exp function.
    return np.exp(self.value)

  def compute_gradients(self):
    # Write your code here
    #compute_gradients() computed using the derivative of the exponential function
    return {self.name : np.exp(self.value)}


Now that we have these classes, let's use them to actually find gradients of complex networks. We finally bring in the idea of a computational graph, which makes it much easier for the gradients to be computed.

This is the image of the example graph that we want you to work with. 





<div>
<img src="https://drive.google.com/uc?id=1VtI1lf85bG8cO1u_8l0D0rqVGhwHQtPK"
 width="500" height="500">
</div>

\begin{equation}
d = 3a \\ 
e = abc \\ 
f = sin(c) \\ 
g = exp(e) \\ 
h = a + d + g + f
\end{equation}

# **Forward Propogation**

In [45]:
def forward_prop(a, b, c) : 
    '''
    Input : real numbers a, b, c.

    Outputs : A dictionary of the values at each node with keys as the names of nodes
    Grads : A dictionary of the gradients at each edge with keys as a pair of nodes 
    
    e.g. Grads["ce"] = ...
    '''
  
    d = 3*a
    e = a*b*c
    #Calculate value of f as the output of a sine function applied on c
    f = Sine("c", c).compute_output()
    # Calculate value of g as the output of an exponential function
    g = Exponential("e", e).compute_output()
    h = a + d + f + g
    # Create a dictionary to store the values of all the variables
    Outputs = {"a": a, "b": b, "c": c, "d": d, "e": e, "f": f, "g": g, "h": h}
    # Create a dictionary to store the gradients of the final output h 
    #with respect to all the input variables
    Grads = {("d", "a"): 3*a,
             ("e", "abc"): (a*b*c),
             ("f", "c"): Sine("c", c).compute_gradients()["c"],
             ("g", "e"): Exponential("e", e).compute_gradients()["e"],
             ("h", "d"): 1,
             ("h", "e"): 1,
             ("h", "f"): 1,
             ("h", "a"): 1,
             ("h", "g"): 1}
    return (Outputs, Grads)

# **Backward Propogation**

Most of the usage of computational graphs in Machine Learning centers around finding the gradients of a particular loss, wrt to all the parameters. Here, your task is to do a simpler version of that.

Use the classes you wrote to calculate the following gradients : 


*   $ \frac{\partial h}{\partial a}$
*   $ \frac{\partial h}{\partial b}$
*   $ \frac{\partial h}{\partial c}$

Use the technique of *backpropogation* to code out the gradients step-wise, along all possible chains of the graph starting from $h$ and ending at $a,b,c$ respectively. 

Don't try to directly get these values without backpropogation, it might be easier here, but with complicated neural networks it wouldn't be :) 



In [46]:
def backward_prop(a, b, c, Outputs, Grads) : 
    '''
    Input : a, b, c (input layer); Outputs (values at each node); Grads (gradients stored at each edge)
    Retuns : result (gradients w.r.t a, b, c)
    '''
    '''To implement the backward propagation, you can use the values stored in the Outputs dictionary along with the gradients stored in the Grads dictionary 
    to calculate the gradients of the final output (h) with respect to the input variables (a, b, c). 
    You can use the chain rule of differentiation to compute the gradients along the different paths in the graph.'''
    #It uses the gradients stored in the Grads dictionary, from previous code
    #which contains the partial derivatives of h with respect to each variable.
    #dh_dacalculated by adding the partial derivatives of h with respect to
    #a and d, and multiplying the partial derivative of h with respect to d by 3.
    dh_da = Grads[("h", "a")] + Grads[("h", "d")] * 3
    #dh_dbb is calculated by multiplying the partial derivative of h 
    #with respect to e by the product of a and c.
    dh_db = Grads[("h", "e")] * (a*c)
    #dh_dc is calculated by adding the product of the partial derivative of h 
    #with respect to e and the product of a and b, and 
    #multiplying the partial derivative of h with respect to f by 
    #the negative sine of c.
    dh_dc = Grads[("h", "e")] * (a*b) + Grads[("h", "f")] * (-np.sin(c))
    #The final result is a list containing the gradients of h with respect to a, b, and c.
    result = [dh_da, dh_db, dh_dc]
    return result





# **Combining both processes**

For the purpose of values, assume that $a = 3, b = 1, c = 2$. Here, we call both forward and backward propogation functions to demonstrate functionality of the functions above.

In [47]:
# Initialisation
a = 3
b = 1
c = 2

# Forward propogation
(Outputs, Grads) = forward_prop(a, b, c)
print(f'Value obtained upon forward propogation : {Outputs["h"]}')

# Backward propogation
result = backward_prop(a, b, c, Outputs, Grads)
print(f'Values of gradients are : {result[0]}, {result[1]}, {result[2]}')

Value obtained upon forward propogation : 416.3380909195608
Values of gradients are : 4, 6, 2.090702573174318


# **Submission Instructions**

Upload this notebook on your github classroom repository by the name Week1.ipynb